# Proyecto Python: clasificador KNN

## Lectura de datos

In [2]:
import numpy as np

In [3]:
# La variable síntomas contiene información de 2000 pacientes.
# La primera columna es la temperatura corporal y la segunda, la concentración de oxígeno
sintomas = np.load('./datasets/sintomas.npy')
print(sintomas)
print(sintomas.shape)

[[35.95150016 97.7293679 ]
 [35.89175305 98.37605467]
 [37.78687243 96.14125368]
 ...
 [36.17146993 98.12967918]
 [36.85791055 96.18333692]
 [37.60602681 95.43248407]]
(2000, 2)


In [4]:
# La variable etiquetas contiene la información sobre si un paciente está enfero - 1 - o san0 - 0.
etiquetas = np.load('./datasets/etiquetas.npy')
print(etiquetas)
print(etiquetas.shape)

[0. 0. 1. ... 0. 1. 1.]
(2000,)


## Desarrollo

In [5]:
# Tu código aquí. Crea tantas celdas como necesites :)

## 1. Separación de los datos en "ENTRENAMIENTO" (80%) y "TEST" (20%)
Cuando hacemos un modelo, necesitamos saber cómo de bueno es para poder comparar y saber qué opción es mejor.

Para ello, dividimos el conjunto en dos: el conjunto de entrenamiento y el de test. 

El conjunto de entrenamiento serán los datos etiquetados que el modelo utilizará, mientras que el conjunto de test servirá para saber cómo de bueno es nuestro modelo, comparando la predicción del modelo con la etiqueta real. Aquí, haremos una separación del 80% de las muestras en el conjunto de entrenamiento y el 20% restante en el de test.

In [6]:
# Creamos la muestra del 80% (muestra entrenamiento) para ambos datasets (sintomas y etiquetas)

#SINTOMAS
entrenamiento_sintomas = sintomas[:1600]
print(entrenamiento_sintomas)
print(entrenamiento_sintomas.size)

#ETIQUETAS
entrenamiento_etiquetas = etiquetas[:1600]
print(entrenamiento_etiquetas)
print(entrenamiento_etiquetas.size)

[[35.95150016 97.7293679 ]
 [35.89175305 98.37605467]
 [37.78687243 96.14125368]
 ...
 [36.72344612 97.1669525 ]
 [37.20902296 98.28675309]
 [35.93575583 97.13610815]]
3200
[0. 0. 1. ... 1. 0. 0.]
1600


In [7]:
# Creamos la muestra del 20% restante (muestra test) para ambos datasets (sintomas y etiquetas)

#SINTOMAS
test_sintomas = sintomas[1600:]
print(test_sintomas)
print(test_sintomas.size)

#ETIQUETAS
test_etiquetas = etiquetas[1600:]
print(test_etiquetas)
print(test_etiquetas.size)

[[ 37.54589609  98.99599913]
 [ 37.86519035  97.77142611]
 [ 36.08056161  97.07239726]
 [ 35.69739467  97.09582731]
 [ 36.57397974  96.25981883]
 [ 36.37474778  96.7277367 ]
 [ 37.50794671  95.57451859]
 [ 35.62869916  97.11295695]
 [ 37.2844403   98.10735852]
 [ 37.29130575  97.90090986]
 [ 36.00897933  98.69844444]
 [ 37.8752362   97.16416169]
 [ 36.8148237   97.34441242]
 [ 37.0659666   96.61651591]
 [ 36.19991569  97.42038932]
 [ 35.80744754  97.85625454]
 [ 37.29348969  97.74727342]
 [ 35.60168461  98.91131991]
 [ 37.37427522  96.63862534]
 [ 37.63086747  98.02617739]
 [ 36.43557543  98.65200381]
 [ 36.74764612  97.56983254]
 [ 36.16127005  97.23174632]
 [ 36.585398    98.85126605]
 [ 37.80777661  96.53033563]
 [ 36.39188408  96.40824168]
 [ 36.64361012  97.92528912]
 [ 37.12750088  96.85532846]
 [ 36.91733834  97.27308091]
 [ 37.93052242  96.73232741]
 [ 38.29643755  95.45900735]
 [ 36.69082004  97.38263886]
 [ 37.58724308  96.07762718]
 [ 37.04784391  96.46037854]
 [ 37.48319123

## 2. Estandarización de los datos

Como la similitud entre datos la vamos a medir con distancias, necesitamos que todas las variables sean parecidas. Para ello, vamos a forzar a nuestras variables a tener media 0 y desviación estándar 1. 

In [8]:
# Calculamos la MEDIA y la DESVIACION TIPICA para la muestra ENTRENAMIENTO (80%) para relativizar los datos

# MEDIA 
media_entrenamiento = np.mean(entrenamiento_sintomas, axis = 0)
print(media_entrenamiento)

# DESVIACION TIPICA
desviacion_entrenamiento = np.std(entrenamiento_sintomas, axis = 0)
print(desviacion_entrenamiento)

# Como comprobación nos imprimimos la longitud de nuestro array para ver que coge la muestra del 80% que queremos
print(len(entrenamiento_sintomas))
print(len(entrenamiento_sintomas[0]))

[36.97694365 97.02655062]
[0.71928337 1.39664731]
1600
2


Aplicamos la formula para la estandarización de la muestra ENTRENAMIENTO (80%) y TEST (20%):

In [9]:
# Calculamos la estandarizacion de la muestra ENTRENAMIENTO y TEST aplicando en una linea de codigo la libreria numpy
entrenamiento_sintomas_estandarizado = (entrenamiento_sintomas - media_entrenamiento) / desviacion_entrenamiento
print(entrenamiento_sintomas_estandarizado)

test_sintomas_estandarizado = (test_sintomas - media_entrenamiento) / desviacion_entrenamiento
print(test_sintomas_estandarizado)

[[-1.42564603  0.50321744]
 [-1.50871081  0.96624541]
 [ 1.12602183 -0.63387294]
 ...
 [-0.35243068  0.1005278 ]
 [ 0.32265353  0.90230544]
 [-1.44753495  0.07844323]]
[[ 7.90999024e-01  1.41012587e+00]
 [ 1.23490510e+00  5.33331131e-01]
 [-1.24621543e+00  3.28262082e-02]
 [-1.77892196e+00  4.96021324e-02]
 [-5.60229704e-01 -5.48980255e-01]
 [-8.37216445e-01 -2.13950880e-01]
 [ 7.38239041e-01 -1.03965548e+00]
 [-1.87442744e+00  6.18669627e-02]
 [ 4.27504194e-01  7.73858859e-01]
 [ 4.37049028e-01  6.26041543e-01]
 [-1.34573433e+00  1.19707660e+00]
 [ 1.24887157e+00  9.85295772e-02]
 [-2.25390927e-01  2.27589166e-01]
 [ 1.23766183e-01 -2.93585011e-01]
 [-1.08028072e+00  2.81988651e-01]
 [-1.62591846e+00  5.94068319e-01]
 [ 4.40085301e-01  5.16037793e-01]
 [-1.91198504e+00  1.34949552e+00]
 [ 5.52399219e-01 -2.77754652e-01]
 [ 9.09132415e-01  7.15733138e-01]
 [-7.52649432e-01  1.16382509e+00]
 [-3.18786086e-01  3.88990054e-01]
 [-1.13400870e+00  1.46920194e-01]
 [-5.44355206e-01  1.306496

In [10]:
# Comprobamos que los datos ENTRENAMIENTO y TEST estan normalizados (la media lo mas cercana a 0, y la desviacion 1)
print(np.mean(entrenamiento_sintomas_estandarizado, axis = 0))
print(np.std(entrenamiento_sintomas_estandarizado, axis = 0))

print(np.mean(test_sintomas_estandarizado, axis = 0))
print(np.std(test_sintomas_estandarizado, axis = 0))


[9.94482274e-16 3.45538008e-14]
[1. 1.]
[ 0.07217519 -0.02005792]
[0.93929206 0.97577193]


## 3. Modelado
En esta parte programaremos la parte de modelado de datos para clasificar a nuestros pacientes. Para ello, deberemos calcular las distancias (similitudes) entre nuestro nuevo paciente - del conjunto de test - y los pacientes que ya sabemos si están sanos o no - del conjunto de entrenamiento. Nuestra predicción será la etiqueta del paciente más cercano.

In [11]:
# Crear el modelo midiendo la distancia entre los puntos en un plano (teorema de Pitagoras) d = √((x_2-x_1)²+(y_2-y_1)²) 

# Recorrer pacientes nuevos (test):
# Para cada uno de los 400 pacientes, 
# tenemos 1600 comparaciones (los de la muestra ENTRENAMIENTO) para ver cuanta distancia hay que coger la minima

# DETERMINAMOS LAS DISTANCIAS DE CADA PACIENTE "TEST" VS "ENTRENAMIENTO"
distancias = np.empty((400,1600), float)

for i in range(len(test_sintomas)):
    distancias[i] = np.sqrt(((entrenamiento_sintomas_estandarizado - test_sintomas_estandarizado[i]) ** 2).sum(axis=1))

print(distancias)
print(distancias.shape)

[[2.39499441 2.3421561  2.0712729  ... 1.73852771 0.6908177  2.60469058]
 [2.66072156 2.77756063 1.17227169 ... 1.64528223 0.98404521 2.7207366 ]
 [0.50345134 0.96962633 2.46414233 ... 0.89634518 1.79369561 0.20642302]
 ...
 [0.41913885 0.42702281 2.65909791 ... 1.03152834 1.44685921 0.78324824]
 [1.67730582 2.06618117 1.29186174 ... 0.72865794 1.58318018 1.45224618]
 [2.82767184 3.18152865 0.56634763 ... 1.74581307 2.11687942 2.62301322]]
(400, 1600)


In [12]:
# Calculamos un array con los indices donde estan las distancias minimas
minimos_indices = np.empty((400,1), float)
minimos_indices = np.argmin(distancias,1)

print(minimos_indices)
print(minimos_indices.shape)

[ 620  135 1179  530   39 1168  344  530 1104  540 1347  636  523  553
  596 1372   35  601  685  537  469 1505 1220  341 1470  933 1035 1462
  482  117  492  123 1193 1457  443  224    3   79 1421  896  740 1585
 1504 1049 1105  713 1586 1151  877  860  980 1284  708 1212  673  310
  341  756  274  595  268  200 1523 1299  146 1421 1122  747 1206   50
  614 1083  156  544 1126 1521 1309 1348  765  506  633  715  114  554
  439 1477  843  124  626  920  121 1342 1069  116  526 1342  639  986
 1256   66  125 1402 1003  240 1547  455  714  604  227  822 1575 1235
  185  229 1228 1391  285  888  556  693  370  472   38  632 1549  681
  307  238  871  446  139 1017  424  214  840  184  470 1092 1511  351
 1535 1249 1292 1160  152 1390  387  537  400 1296   29 1150  424 1369
 1390 1251  990 1026  315  666  863 1227  174 1184  964  523 1379 1073
 1591  208 1270  842 1527 1509 1149  847  419  528  906  452  749 1194
 1498  443 1257 1056 1363 1247  317 1569 1082 1425  874 1424  858 1286
  466 

In [13]:
# Utilizamos el array generado de las posiciones con los minimos para sacar la prediccion de cada paciente test
prediccion_posiciones = np.empty((400,1), float)

prediccion_posiciones = entrenamiento_etiquetas [minimos_indices]

print(prediccion_posiciones)
print(prediccion_posiciones.shape)

[0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0.
 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.

## 4. Rendimiento del modelo
Tras obtener nuestro modelo, queremos saber cómo de efectivo es a la hora de clasificar a nuestros pacientes. Para ello, vamos a calcular el porcentaje de veces que nuestro modelo acierta la etiqueta correcta.

In [14]:
# De la muestra test nos imprimimos las etiquetas simplemente para ver si nos va coincidiendo con lo que acabamos de sacar de la prediccioon
print(test_etiquetas)

[0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1.
 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.

In [15]:
# Comparamos test_etiquetas con el array generado en prediccion
aciertos = 0
for i in range(len(prediccion_posiciones)):
    if prediccion_posiciones [i] == test_etiquetas[i]:
        aciertos = aciertos + 1
rendimiento  = aciertos / len(test_etiquetas)
print('La efectividad del modelo es de un', rendimiento *100, '%')

La efectividad del modelo es de un 87.5 %
